In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import joblib

# =========================
# Paths
# =========================
MODEL_PATH = "currentOlsSolution.pkl"
FEATURES_PATH = "ols_features.pkl"
ACTIVATION_PATH = "../../data/activation_data.csv"

# =========================
# Load model and feature list
# =========================
ols_model = joblib.load(MODEL_PATH)
ols_features = joblib.load(FEATURES_PATH)

activation_df = pd.read_csv(ACTIVATION_PATH)

# =========================
# Feature engineering (MUST match training)
# =========================
activation_df["Date"] = pd.to_datetime(activation_df["Date"])
activation_df["Year"]  = activation_df["Date"].dt.year
activation_df["Month"] = activation_df["Date"].dt.month
activation_df["Day"]   = activation_df["Date"].dt.day

# Drop non-features
# =========================
# Feature engineering (EXACTLY like training)
# =========================
activation_df["Price_Range"] = (
    activation_df["High_Price"] - activation_df["Low_Price"]
)

activation_df["Return_Volatility"] = (
    activation_df["Daily_Return_Pct"] * activation_df["Volatility_Range"]
)

activation_df["Volume_Impact"] = (
    activation_df["Volume"] * activation_df["Daily_Return_Pct"]
)

X_activation = activation_df.drop(
    columns=["Date", "Close_Price"],
    errors="ignore"
)


# =========================
# Align EXACTLY to training features
# =========================
# Remove constant from feature list
feature_cols = [f for f in ols_features if f != "const"]

# Reindex to training order
X_activation = X_activation.reindex(
    columns=feature_cols,
    fill_value=0
)

# Add constant ONCE
X_activation = sm.add_constant(X_activation, has_constant="add")

# Final strict ordering
X_activation = X_activation[ols_features].astype(np.float64)

# =========================
# Prediction (REAL SCALE)
# =========================
ols_pred = ols_model.predict(X_activation).values.flatten()

activation_df["Predicted_Close_Price"] = ols_pred

print(
    activation_df[["Close_Price", "Predicted_Close_Price"]].head()
)


=== OLS Activation Sanity Check ===
min : 11.180423348981446
max : 11.180423348981446
mean: 11.180423348981446
   Close_Price  Predicted_Close_Price
0        11.62              11.180423


Create this table once (code or manual):

Model	RMSE	R²
ANN	lower	higher
OLS	higher	lower